# Consistent scores

A "consistent scoring function" is a scoring function where following a forecast directive will optimise the forecaster's expected score. It is important to use consistent scoring functions to avoid the "forecaster's dilemma" ([Lerch et al., 2017](https://projecteuclid.org/journals/statistical-science/volume-32/issue-1/Forecasters-Dilemma-Extreme-Events-and-Forecast-Evaluation/10.1214/16-STS588.full)) (i.e., should a forecaster issue an honest forecast or should they issue one that optimises their expected score?).


The MSE is a consistent score for forecasting the mean and the MAE is a consistent score for forecasting the median. But did you know that there are a whole family of scores that are consistent for prediciting the mean, median, or a quantile? 

Score's consistent scoring module provides access to these families of consistent scores that can we used to emphasise predictive performance across desired decision thresholds.

### Example 1. A score consistent with the mean
Let's jump into an example that illustrates a scoring function that is consistent with predicting the mean value using scores. **-Add interpretation here**

Medians relate to quantiles in an analogous way to how the mean relates to [expectiles](https://en.wikipedia.org/wiki/Expectile).

Our consistent scoring function for expectiles is 

 $$ S(x, y) =
    \begin{cases}
    (1 - \alpha)(\phi(y) - \phi(x) - \phi'(x)(y-x)), & y < x \\
    \alpha(\phi(y) - \phi(x) - \phi'(x)(y-x)), & x \leq y
    \end{cases}
$$

Where:
- $x$ is the forecast
- $y$ is the observation
- $\alpha$ is the expectile level
- $\phi$ is a [convex function](https://en.wikipedia.org/wiki/Convex_function) of a single variable
- $\phi'$ is the [subderivative](https://en.wikipedia.org/wiki/Subderivative) of $\phi$. The subderivative is a generalisation of the derivative for convex functions and coincides with the derivative when the convex function is differentiable
- $S(x,y)$ is the score.

To make the expectile the consistent with the mean, set $\alpha=0.5$.

In our example let's set $\phi(z) = e^z$. This means that when we take the subderivative of $\phi(z)$, we also have $\phi'(z) = e^z$.

Our scoring function then is 
$$ S(x, y) = 0.5(e^y - e^x - e^x(y-x))$$


In [7]:
import numpy as np
x = 9
y = 10
0.5 * (np.exp(y)-np.exp(x) - np.exp(x)*(y-x))

2910.1489698279747